In [1]:
import PIL
from PIL import Image
import numpy as np
import os 
import random
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/a

In [2]:
dirs = ['/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/no_porosity',
        '/media/jan/Volume_big/datasets/Second_try/Grid_size_874_binary/3layer/porosity']

length = 2000
image_paths = []
labels = []
data = []

In [3]:
height = 128
width = 128
depth = 3
inputShape = (height, width, depth)

In [4]:
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)
        cur_paths.append(full_path)
        
    random.seed(42)
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length]
    
    image_paths.append(cur_paths)        

image_paths = [val for sublist in image_paths for val in sublist]
random.seed(42)
random.shuffle(image_paths)

######
for path in image_paths:
    label = path.split(os.path.sep)[-2]
    labels.append(label)
    
    array = np.load(path)
    img = Image.fromarray(array)
    img = img.resize((height, width), PIL.Image.LANCZOS)
    array_resized_flattened = np.array(img)
    data.append(array_resized_flattened)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = np.hstack((trainY, 1- trainY))

testY = lb.transform(testY)
testY = np.hstack((testY, 1- testY))

In [5]:
def create_model():
    
    model = Sequential()
    model.add(Conv2D(3,(6,6), padding="same", input_shape = inputShape))   #, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(6, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(2))
    model.add(Activation("sigmoid"))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model 

In [7]:
trainY.shape

(3000, 2)

In [8]:
trainY[1]

array([1, 0])

In [9]:
model = KerasClassifier(build_fn=create_model)

In [10]:
batch_size = [8, 16, 32]
epochs = [20, 40, 60 ]

param_grid = dict(batch_size=batch_size, epochs=epochs)

In [11]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)

In [12]:
grid_result = grid.fit(trainX, trainY)

Epoch 1/20
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6652 - acc: 0.6283
Epoch 2/20
1688/2000 [========================>.....] - ETA: 0s - loss: 0.6124 - acc: 0.6795

KeyboardInterrupt: 

In [25]:
grid_result.cv_results_

{'mean_fit_time': array([ 73.68156854, 143.68014566, 217.118457  ,  47.42368134,
         89.58552599, 134.55436039,  37.21461654,  66.33197959,
         95.801958  ]),
 'std_fit_time': array([0.21188654, 1.14867643, 1.70199721, 0.7021431 , 0.20612979,
        1.95143775, 0.63712292, 1.22583816, 0.36364875]),
 'mean_score_time': array([2.58037845, 2.87928422, 3.28858956, 3.28682788, 3.70697006,
        4.11455655, 4.35195009, 5.08571426, 5.49253813]),
 'std_score_time': array([0.06545984, 0.09177454, 0.23222692, 0.08418371, 0.06928528,
        0.15753874, 0.10290539, 0.35226705, 0.16243076]),
 'param_batch_size': masked_array(data=[8, 8, 8, 16, 16, 16, 32, 32, 32],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[20, 40, 60, 20, 40, 60, 20, 40, 60],
              mask=[False, False, False, False, False, False, False, False,
                    F

In [26]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.705167 using {'batch_size': 8, 'epochs': 20}
0.705167 (0.012795) with: {'batch_size': 8, 'epochs': 20}
0.703833 (0.007192) with: {'batch_size': 8, 'epochs': 40}
0.692833 (0.008270) with: {'batch_size': 8, 'epochs': 60}
0.686167 (0.010881) with: {'batch_size': 16, 'epochs': 20}
0.684500 (0.010985) with: {'batch_size': 16, 'epochs': 40}
0.700333 (0.006196) with: {'batch_size': 16, 'epochs': 60}
0.696667 (0.012250) with: {'batch_size': 32, 'epochs': 20}
0.687000 (0.010206) with: {'batch_size': 32, 'epochs': 40}
0.699833 (0.010781) with: {'batch_size': 32, 'epochs': 60}
